# Conjugate Gradients with operators

Written by H. Tagare 2/8/2024

The operators are defined in the folder optlib
The CG algorithm is defined in the folder c_grad
Both have to imported to get everything to work

In [2]:
import numpy as np
import optlib.operators as op
import optlib.c_grad as cg

### Test 1: Simple test without noise
#### The operator A is (MO+R). Task is to recover a vector. No regularization on CG

In [3]:
M=op.matrix_op(np.array(((1, 2, 3),(0, 1, 0),(2, 3,1))))
O=op.scalar_prod_op(2.0)
R=op.scalar_prod_op(0.1)
A=op.add_op(op.composite_op(M,O),R) 

x=np.array((1,0.5,1)).reshape(-1,1) #Input x
y=A.forward(x) #Create noiseless output by applying A
print(f"x={x}\n y={y}")

x0=np.zeros_like(x) #Initial value for CG
x,flag=cg.c_grad(y,A,x0)
print(f"x={x} \n flag={flag}")

x=[[1. ]
 [0.5]
 [1. ]]
 y=[[10.1 ]
 [ 1.05]
 [ 9.1 ]]
CG: step=1 res_norm=3.234651420766811 
CG: step=2 res_norm=0.7480204131885724 
CG: step=3 res_norm=6.8985257123446856e-12 
x=[[1. ]
 [0.5]
 [1. ]] 
 flag=False


### Test 2: x is matrix, y contains noise, CG uses regularizer

In [4]:
M=op.matrix_op(np.array(((1, 2, 3),(0, 1, 0),(2, -3,1))))
O=op.scalar_prod_op(2.0)
R=op.scalar_prod_op(0.1)

A=op.add_op(op.composite_op(M,O),R)

x=np.transpose(np.array(((1,0.5,1),(0,1,0))))
y=A.forward(x)
y=y+0.1*np.random.normal(size=y.shape)
print(f"x={x}\n y={y}")

x0=np.zeros_like(x)
x,flag=cg.c_grad(y,A,x0,B=R,f_tol=1e-6)
print(f"x={x} \n flag={flag}")


x=[[1.  0. ]
 [0.5 1. ]
 [1.  0. ]]
 y=[[10.11272192  3.924224  ]
 [ 0.91623733  2.03982036]
 [ 3.00978323 -6.17381145]]
CG: step=1 res_norm=8.254539085976411 
CG: step=2 res_norm=0.15339979165680256 
CG: step=3 res_norm=1.693502937786407e-13 
x=[[ 0.82286805 -0.11322942]
 [ 0.43475462  0.96988142]
 [ 1.10738359  0.04704378]] 
 flag=False


### CG with fft

In [5]:
x=np.random.normal(size=(4,4))
print(x)
A=op.real_fftn_op()
y=A.forward(x)
y=y+0.2*np.random.normal(size=y.shape)
#print(f"x={x}\n y={y}")
print(f"x={x}")

x0=np.zeros_like(x)
R=op.scalar_prod_op(0.1)
x,_=cg.c_grad(y,A,x0,B=R)
print(x)

[[-0.74847011  0.0252821   1.04943368  0.02033035]
 [-1.27323234  0.35880291  0.4664044  -1.79830664]
 [-0.32549423  1.4674783   1.28876532 -0.09549174]
 [ 0.86345455 -1.32773834  1.08637846 -0.01367917]]
x=[[-0.74847011  0.0252821   1.04943368  0.02033035]
 [-1.27323234  0.35880291  0.4664044  -1.79830664]
 [-0.32549423  1.4674783   1.28876532 -0.09549174]
 [ 0.86345455 -1.32773834  1.08637846 -0.01367917]]
CG: step=1 res_norm=2.0407899217551515e-16 
[[-0.6411034   0.07410848  1.00527908  0.06920576]
 [-1.24513369  0.43017781  0.43819651 -1.71306912]
 [-0.41067715  1.45547539  1.28818815 -0.0920197 ]
 [ 0.87039788 -1.24715991  1.05203222  0.06138367]]


In [6]:
x=np.random.normal(size=(4,4))+1j*np.random.normal(size=(4,4))
print(x)
A=op.fftn_op()
y=A.forward(x)
y=y+0.2*np.random.normal(size=y.shape)
#print(f"x={x}\n y={y}")
print(f"x={x}")

x0=np.zeros_like(x)
R=op.scalar_prod_op(0.1)
x,_=cg.c_grad(y,A,x0,B=R)
print(x)

[[ 0.86385648-0.31185353j  0.30963617+0.9197518j  -1.82486437-0.160047j
   1.05191509-0.6500287j ]
 [ 1.39106785+1.80722323j -0.26862791-1.65648538j  1.00808291+0.12334677j
   0.47920033+0.78423757j]
 [-1.58596583+0.38815376j  0.87893735-0.14099236j  1.17247354+1.08290501j
  -0.75271663+0.69859992j]
 [ 0.5032992 -0.63724547j  1.43416671+1.85455308j  0.29014939+1.36207753j
   0.04378243+0.46808921j]]
x=[[ 0.86385648-0.31185353j  0.30963617+0.9197518j  -1.82486437-0.160047j
   1.05191509-0.6500287j ]
 [ 1.39106785+1.80722323j -0.26862791-1.65648538j  1.00808291+0.12334677j
   0.47920033+0.78423757j]
 [-1.58596583+0.38815376j  0.87893735-0.14099236j  1.17247354+1.08290501j
  -0.75271663+0.69859992j]
 [ 0.5032992 -0.63724547j  1.43416671+1.85455308j  0.29014939+1.36207753j
   0.04378243+0.46808921j]]
CG: step=1 res_norm=7.714352551028867e-16 
[[ 0.82050031-0.30876587j  0.28970463+0.92776103j -1.74056791-0.15846237j
   1.02463425-0.66070845j]
 [ 1.35597518+1.80929633j -0.20645877-1.6642797j